---
title: アップサンプリング + パディング処理の流れ
date: 2025-06-15
categories: [ML]
---

## アップサンプリング + パディング処理の流れ

### 1. 入力テンソルの準備

3次元の NumPy テンソル `x`（形状: `(B, T, C)`）を作成します。  
ここではインデックスを使って中身を可視化しやすくしています。

---

### 2. アップサンプリング処理（ratio倍）

`x[:, :, None, :].repeat(1, 1, ratio, 1)` によって、時間軸の各ステップを `ratio` 倍に拡張します。

- `x[:, :, None, :]` → 次元を追加して `(B, T, 1, C)`
- `.repeat(...)` → `(B, T, ratio, C)` に拡張
- `.reshape(B, T * ratio, C)` → 時間軸を1本化

結果、時間方向に `T * ratio` 長のテンソルが得られます。

---

### 3. 足りない分のパディング

指定の長さ `frame_size` に満たない場合、**最後のステップの値を複製**して補います。

- `y[:, -1:, :]` で最後のフレームを取り出し
- `.repeat(...)` で必要数だけ繰り返し
- `torch.cat([y, padding], dim=1)` で結合

長さがすでに `frame_size` 以上なら、先頭から `frame_size` 分だけを切り出します。

---

### 最終出力

この処理で、任意の入力テンソルを：

- **指定の時間長 (`frame_size`) に統一**
- **滑らかに拡張しつつ、端は自然に補完**

することができます。

---

**用途の例**:
- 音声や動画の時系列データ整形
- Transformer 系への入力前の整形
- ラベル付きデータのアライメント補正

---


In [22]:
import numpy as np
import torch

In [27]:
# セル 2: パラメータと元データ作成
B, T, C = 2, 3, 4  # バッチサイズ、時間長、チャネル数
ratio = 3         # アップサンプリング倍率
frame_size = 11          # 最終的な長さにしたいサイズ

# 見やすいようにインデックス付きデータを作成
x = torch.arange(B * T * C).reshape(B, T, C)
print("元のxの shape:", x.shape)
print(x)

元のxの shape: torch.Size([2, 3, 4])
tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])


In [29]:
# セル 3: ratio 倍に拡張（アップサンプリング）
x_expanded = x[:, :, None, :].repeat(1, 1, ratio, 1)
print("拡張後の shape:", x_expanded.shape)
print(x_expanded)

拡張後の shape: torch.Size([2, 3, 3, 4])
tensor([[[[ 0,  1,  2,  3],
          [ 0,  1,  2,  3],
          [ 0,  1,  2,  3]],

         [[ 4,  5,  6,  7],
          [ 4,  5,  6,  7],
          [ 4,  5,  6,  7]],

         [[ 8,  9, 10, 11],
          [ 8,  9, 10, 11],
          [ 8,  9, 10, 11]]],


        [[[12, 13, 14, 15],
          [12, 13, 14, 15],
          [12, 13, 14, 15]],

         [[16, 17, 18, 19],
          [16, 17, 18, 19],
          [16, 17, 18, 19]],

         [[20, 21, 22, 23],
          [20, 21, 22, 23],
          [20, 21, 22, 23]]]])


In [30]:
# セル 4: reshape して時間方向を1本化
x_reshaped = x_expanded.reshape(B, T * ratio, C)
print("reshape後の shape:", x_reshaped.shape)
print(x_reshaped)

reshape後の shape: torch.Size([2, 9, 4])
tensor([[[ 0,  1,  2,  3],
         [ 0,  1,  2,  3],
         [ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 4,  5,  6,  7],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [ 8,  9, 10, 11],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [12, 13, 14, 15],
         [12, 13, 14, 15],
         [16, 17, 18, 19],
         [16, 17, 18, 19],
         [16, 17, 18, 19],
         [20, 21, 22, 23],
         [20, 21, 22, 23],
         [20, 21, 22, 23]]])


In [31]:
# セル 5: PyTorchテンソルに変換して padding 処理
y = x_reshaped

# 足りない長さを補う（最後のステップを複製）
if y.shape[1] < frame_size:
    padding = y[:, -1:, :].repeat(1, frame_size - y.shape[1], 1)
    y_padded = torch.cat([y, padding], dim=1)
else:
    y_padded = y[:, :frame_size, :]  # もし長すぎたら切り詰める

print("最終 y_padded の shape:", y_padded.shape)
print(y_padded)

最終 y_padded の shape: torch.Size([2, 11, 4])
tensor([[[ 0,  1,  2,  3],
         [ 0,  1,  2,  3],
         [ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 4,  5,  6,  7],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [ 8,  9, 10, 11],
         [ 8,  9, 10, 11],
         [ 8,  9, 10, 11],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [12, 13, 14, 15],
         [12, 13, 14, 15],
         [16, 17, 18, 19],
         [16, 17, 18, 19],
         [16, 17, 18, 19],
         [20, 21, 22, 23],
         [20, 21, 22, 23],
         [20, 21, 22, 23],
         [20, 21, 22, 23],
         [20, 21, 22, 23]]])
